In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from google.cloud import bigquery

In [2]:
def get_sp500_tickers():
    return pd.read_csv("tickers.csv", header=None)[0].tolist()


# Download previous day's data
def download_previous_day_data(tickers):
    """
    Downloads daily data for the tickers from the previous day.
    """
    # start_date = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
    # end_date = (datetime.now() - timedelta(days=0)).strftime("%Y-%m-%d")

    current_date = datetime.now() - timedelta(days=1)
    while not pd.Timestamp(current_date).isoweekday() in range(
        1, 6
    ):  # Monday (1) to Friday (5)
        current_date -= timedelta(days=1)

    start_date = current_date.strftime("%Y-%m-%d")
    end_date = (current_date + timedelta(days=1)).strftime("%Y-%m-%d")

    data = yf.download(
        tickers=" ".join(tickers),
        start=start_date,
        end=end_date,
        interval="1d",
        group_by="ticker",
        threads=True,
    )
    if data.empty:
        print(f"No data available for {start_date}.")
        return pd.DataFrame()  # Returns an empty DataFrame if no data is available
    # Restructure the data
    data = data.stack(level=0, future_stack=True).reset_index()
    data.columns = [
        "Date",
        "Ticker",
        "Open",
        "High",
        "Low",
        "Close",
        "Adj Close",
        "Volume",
    ]
    return data


# Download data for all tickers
def download_sp500_data(tickers, start_date="2000-01-01", end_date=None):
    """
    Downloads daily data for all S&P 500 tickers.
    """
    # Fetch the data via yfinance
    data = yf.download(
        tickers=" ".join(tickers),
        start=start_date,
        end=end_date,
        interval="1d",
        group_by="ticker",  # Organizes the data by ticker
        threads=True,  # Parallel download
    )

    data = data.stack(level=0, future_stack=True).reset_index()  # Make tickers a column
    data.columns = [
        "Date",
        "Ticker",
        "Open",
        "High",
        "Low",
        "Close",
        "Adj Close",
        "Volume",
    ]

    return data


# Load data into BigQuery
def load_data_to_bigquery(df, table_id, project_id):
    """
    Loads a DataFrame into a BigQuery table.

    Arguments:
        df : pandas.DataFrame containing the data to load.
        table_id : Full ID of the BigQuery table (e.g., dataset.table_name).
        project_id : Google Cloud project ID.
    """
    client = bigquery.Client(project=project_id)

    # Convert the data to BigQuery format
    job = client.load_table_from_dataframe(df, table_id)

    # Wait for the job to complete
    job.result()
    print(f"Data has been loaded into {table_id}")


# Load data into a temporary table
def load_to_temp_table(client, df, temp_table_id):
    """
    Loads data into a temporary BigQuery table.
    """
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE"  # Overwrites existing data
    )
    job = client.load_table_from_dataframe(df, temp_table_id, job_config=job_config)
    job.result()
    print(f"Data loaded into temporary table {temp_table_id}")


# Merge data with the main table
def merge_into_main_table(client, temp_table_id, main_table_id):
    """
    Merges data from the temporary table into the main table.
    """
    query = f"""
    MERGE `{main_table_id}` AS main
    USING `{temp_table_id}` AS temp
    ON main.Date = temp.Date AND main.Ticker = temp.Ticker
    WHEN NOT MATCHED THEN
      INSERT (Date, Ticker, Open, High, Low, Close, `Adj Close`, Volume)  -- `Adj Close` with a space
      VALUES (temp.Date, temp.Ticker, temp.Open, temp.High, temp.Low, temp.Close, temp.`Adj Close`, temp.Volume)
    """
    job = client.query(query)
    job.result()
    print(f"Data merged into main table {main_table_id}")


def fill_table():
    # Step 1: Fetch tickers
    print("Downloading S&P 500 tickers...")
    sp500_tickers = get_sp500_tickers()
    print(f"Number of tickers fetched: {len(sp500_tickers)}")

    # Step 2: Download data
    print("Downloading daily data...")
    sp500_data = download_sp500_data(sp500_tickers, start_date="2000-01-01")

    # # Step 3: Load into BigQuery
    # print("Loading data into BigQuery...")
    # PROJECT_ID = "quant-dev-442615"  # Replace with your project ID
    # DATASET_ID = "financial_data"  # Replace with your dataset name
    # TABLE_ID = "sp500_data"  # Table name

    # # Load data
    # load_data_to_bigquery(
    #     sp500_data,
    #     table_id=f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}",
    #     project_id=PROJECT_ID,
    # )
    return sp500_data


def add_daily():
    # Step 1: Configuration
    PROJECT_ID = "quant-dev-442615"
    DATASET_ID = "financial_data"
    MAIN_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.sp500_data"
    TEMP_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.temp_sp500_data"

    # Fetch tickers
    print("Downloading S&P 500 tickers...")
    tickers = get_sp500_tickers()

    # Step 2: Download data
    print("Downloading data for the previous day...")
    sp500_data = download_previous_day_data(tickers)
    if sp500_data.empty:
        print("No data available. Execution stopped.")
        return  # Stop execution if no data is available
    sp500_data["Date"] = pd.to_datetime(sp500_data["Date"]).dt.date
    # Convert the Volume column to integer
    sp500_data["Volume"] = sp500_data["Volume"].fillna(0).astype(int)

    # Step 3: Load into BigQuery
    client = bigquery.Client(project=PROJECT_ID)
    print("Loading data into a temporary table...")
    load_to_temp_table(client, sp500_data, TEMP_TABLE_ID)

    # Step 4: Merge with the main table
    print("Merging data with the main table...")
    merge_into_main_table(client, TEMP_TABLE_ID, MAIN_TABLE_ID)

In [3]:
df = fill_table()

Number of tickers fetched: 519


[*********************100%***********************]  519 of 519 completed


ValueError: Length mismatch: Expected axis has 7 elements, new values have 8 elements

In [3]:
# Fetch the data via yfinance
start_date = "2000-01-01"
end_date = None
tickers = get_sp500_tickers()
data = yf.download(
    tickers=" ".join(tickers),
    start=start_date,
    end=end_date,
    interval="1d",
    group_by="ticker",  # Organizes the data by ticker
    threads=True,  # Parallel download
)

[*********************100%***********************]  519 of 519 completed

1 Failed download:
['TSCO']: ConnectionError(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))


In [ ]:
df_work = data.stack(level=0, future_stack=True).reset_index()

In [ ]:
data[["Ticker", "Date", "Open", "High", "Low", "Close", "Volume"]]

Price,Ticker,Date,Open,High,Low,Close,Volume
0,BTC-USD,2000-01-03,NaN,NaN,NaN,NaN,NaN
1,LYV,2000-01-03,NaN,NaN,NaN,NaN,NaN
2,HLT,2000-01-03,NaN,NaN,NaN,NaN,NaN
3,AVGO,2000-01-03,NaN,NaN,NaN,NaN,NaN
4,INCY,2000-01-03,32.656250,44.875000,32.625000,44.828125,3834800.0
...,...,...,...,...,...,...,...
3944914,APA,2025-01-14,24.219999,24.604000,24.139999,24.461901,828115.0
3944915,SCHW,2025-01-14,NaN,NaN,NaN,NaN,NaN
3944916,SBUX,2025-01-14,94.000000,94.889999,93.730003,94.070000,1180889.0
3944917,CDNS,2025-01-14,293.755005,295.329987,291.910004,292.500000,148510.0


In [28]:
from datetime import datetime, timedelta

In [29]:
(datetime.today() - timedelta(1)).strftime("%Y-%m-%d")

'2025-01-13'

In [10]:
data.xs("AAPL", axis=1, level="Ticker").reset_index()

Price,Date,Open,High,Low,Close,Volume
0,2000-01-03,0.789884,0.847313,0.765877,0.843077,535796800.0
1,2000-01-04,0.815304,0.833191,0.762111,0.771997,512377600.0
2,2000-01-05,0.781411,0.832720,0.775762,0.783293,778321600.0
3,2000-01-06,0.799299,0.805889,0.715508,0.715508,767972800.0
4,2000-01-07,0.726806,0.760699,0.719275,0.749401,460734400.0
...,...,...,...,...,...,...
7596,2025-01-10,240.009995,240.160004,233.000000,236.850006,61710900.0
7597,2025-01-11,NaN,NaN,NaN,NaN,NaN
7598,2025-01-12,NaN,NaN,NaN,NaN,NaN
7599,2025-01-13,233.529999,234.669998,229.720001,234.399994,49555000.0


In [33]:
test = pd.read_csv("sp500_data_test.csv")

In [34]:
test.tail(50)

,Date,Ticker,Open,High,Low,Close,Volume
3944350,2025-01-13,JNPR,38.290001,38.560001,38.250000,38.490002,1700600.0
3944351,2025-01-13,COF,175.479996,180.509995,174.750000,180.330002,4205300.0
3944352,2025-01-13,AMD,115.279999,117.480003,114.410004,117.320000,38975200.0
3944353,2025-01-13,TSN,54.840000,55.980000,54.480000,55.630001,2507800.0
3944354,2025-01-13,USB,47.540001,48.410000,47.410000,48.389999,9164200.0
3944355,2025-01-13,^GDAXI,20157.240234,20176.109375,20025.279297,20132.849609,56263600.0
3944356,2025-01-13,MNST,49.990002,50.189999,49.389999,49.990002,5633300.0
3944357,2025-01-13,TPL,1300.000000,1339.089966,1287.050049,1300.569946,155300.0
3944358,2025-01-13,CTVA,57.200001,60.029999,57.200001,59.930000,3845400.0
3944359,2025-01-13,WMT,92.120003,92.430000,91.099998,91.529999,18607900.0
